In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from pytorch_tabular import TabularModel
from pytorch_tabular.models import TabTransformerConfig
from sklearn.preprocessing import (
    LabelEncoder,
    MinMaxScaler
)
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
    ExperimentConfig
)
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
import torch


In [9]:
data = pd.read_csv("reshaped_data_final_corrected.csv")
df = pd.DataFrame(data)
df.head()

,championName,tier,position,teamId,win,kills,deaths,assists,totalDamageDealtToChampions,visionScore,...,pentaKills.9,neutralMinionsKilled.9,magicDamageDealt.9,magicDamageDealtToChampions.9,magicDamageTaken.9,wardsPlaced.9,gameMode.9,gameVersion.9,gameDuration.9,gameId.9
0,Malzahar,GOLD,MIDDLE,100,True,5,3,3,20429,19,...,0,136,993,892,8459,1,CLASSIC,13.6.499.7758,1415,6432690956
1,Morgana,GOLD,JUNGLE,100,True,5,7,20,29274,88,...,0,8,171895,45746,34326,6,CLASSIC,13.6.499.7758,2288,6432696459
2,Yasuo,BRONZE,BOTTOM,100,True,11,16,14,41681,12,...,0,11,185530,32666,13844,11,CLASSIC,13.6.499.7758,2267,6432704556
3,Kaisa,GOLD,BOTTOM,100,True,11,5,6,36301,34,...,0,24,153672,24773,8447,13,CLASSIC,13.6.499.7758,2320,6432704874
4,Warwick,SILVER,TOP,100,False,7,8,2,12299,9,...,0,0,114107,20033,10507,5,CLASSIC,13.6.499.7758,1489,6432709941


In [10]:
df['target'] = df['win'].apply(lambda x: 1 if x == True else 0)
df.drop(columns=['win', 'win.1', 'win.2', 'win.3', 'win.4', 'win.5', 'win.6', 'win.7', 'win.8', 'win.9'])


,championName,tier,position,teamId,kills,deaths,assists,totalDamageDealtToChampions,visionScore,inhibitorKills,...,neutralMinionsKilled.9,magicDamageDealt.9,magicDamageDealtToChampions.9,magicDamageTaken.9,wardsPlaced.9,gameMode.9,gameVersion.9,gameDuration.9,gameId.9,target
0,Malzahar,GOLD,MIDDLE,100,5,3,3,20429,19,0,...,136,993,892,8459,1,CLASSIC,13.6.499.7758,1415,6432690956,1
1,Morgana,GOLD,JUNGLE,100,5,7,20,29274,88,0,...,8,171895,45746,34326,6,CLASSIC,13.6.499.7758,2288,6432696459,1
2,Yasuo,BRONZE,BOTTOM,100,11,16,14,41681,12,0,...,11,185530,32666,13844,11,CLASSIC,13.6.499.7758,2267,6432704556,1
3,Kaisa,GOLD,BOTTOM,100,11,5,6,36301,34,0,...,24,153672,24773,8447,13,CLASSIC,13.6.499.7758,2320,6432704874,1
4,Warwick,SILVER,TOP,100,7,8,2,12299,9,0,...,0,114107,20033,10507,5,CLASSIC,13.6.499.7758,1489,6432709941,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299305,Yone,SILVER,MIDDLE,100,7,8,5,23480,15,1,...,100,732,583,12351,10,CLASSIC,14.10.585.8823,1661,7075318881,1
299306,Yuumi,SILVER,UTILITY,100,2,1,14,12830,19,0,...,0,432,112,8586,4,CLASSIC,14.10.585.8823,1287,7075331682,1
299307,Mordekaiser,SILVER,TOP,100,3,4,5,27442,10,0,...,0,65012,11923,5952,12,CLASSIC,14.10.585.8823,1577,7075333832,1
299308,Viktor,SILVER,MIDDLE,100,0,4,0,12227,13,0,...,0,12471,6322,858,24,CLASSIC,14.10.585.8823,1655,7075373693,0


In [11]:
# 가상의 데이터 생성

# 범주형 및 수치형 데이터 처리
categorical_columns = [ col for col in df.columns if df[col].dtype == "object"]
numerical_columns = [ col for col in df.columns if df[col].dtype != "object"]
encoder = OrdinalEncoder()
df[categorical_columns] = encoder.fit_transform(df[categorical_columns])
# 범주형 데이터 인코딩
# for col in categorical_columns:
#     # input_seq =torch.LongTensor([self.word_to_int[word]for word in sample[:-1]])
#     # le = LabelEncoder()
#     df[col] = torch.LongTensor([word in word for word in df[col]])

    # df[col] = le.fit_transform(df[col])

# 수치형 데이터 스케일링
scaler = StandardScaler()
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

# df[numerical_columns]
train_df, valid_df = train_test_split(df, test_size=0.2, random_state=42)
print(train_df.head())
print(valid_df.head())
print("Train DF Shape:", train_df.shape)
print("Valid DF Shape:", valid_df.shape)


c:\Users\a\AppData\Local\pypoetry\Cache\virtualenvs\tabulr_transformers-5PYuSfOC-py3.11\Lib\site-packages\sklearn\utils\extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\a\AppData\Local\pypoetry\Cache\virtualenvs\tabulr_transformers-5PYuSfOC-py3.11\Lib\site-packages\sklearn\utils\extmath.py:1142: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
c:\Users\a\AppData\Local\pypoetry\Cache\virtualenvs\tabulr_transformers-5PYuSfOC-py3.11\Lib\site-packages\sklearn\utils\extmath.py:1162: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


        championName  tier  position  teamId       win     kills    deaths  \
187981          33.0   1.0       3.0     0.0  0.985821  1.734062  0.371929   
67977          148.0   0.0       2.0     0.0 -1.014383 -0.592623 -0.228340   
64272          127.0   0.0       2.0     0.0 -1.014383  0.464961  0.672063   
262999          68.0   1.0       2.0     0.0  0.985821 -0.381106 -0.828610   
120108          72.0   3.0       0.0     0.0  0.985821  0.041927  1.872602   

         assists  totalDamageDealtToChampions  visionScore  ...  \
187981 -0.232525                     1.446699     0.224572  ...   
67977  -0.961786                    -1.131530    -0.708928  ...   
64272   0.679051                     0.542272     0.224572  ...   
262999 -0.779471                    -0.847232    -0.475553  ...   
120108 -0.597155                     0.279939    -0.522228  ...   

        neutralMinionsKilled.9  magicDamageDealt.9  \
187981               -0.401625           -0.600923   
67977               

In [12]:
# 데이터 설정
data_config = DataConfig(
    target=['win'],             # 타겟 변수
    continuous_cols=numerical_columns,    # 수치형 변수
    categorical_cols=categorical_columns  # 범주형 변수
)

# 모델 설정
model_config = TabTransformerConfig(
    task="classification",
    learning_rate=1e-3,
)

# 학습 설정
trainer_config = TrainerConfig(
    auto_lr_find=True,  # Learning rate 자동 탐색
    batch_size=min(16, len(train_df)),
    max_epochs=100,
    accelerator='gpu'
)

# 모델 초기화 및 학습
tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=OptimizerConfig(),
    trainer_config=trainer_config,
    verbose=True
)
train_df


2024-05-20 11:00:22,421 - {pytorch_tabular.tabular_model:140} - INFO - Experiment Tracking is turned off

,championName,tier,position,teamId,win,kills,deaths,assists,totalDamageDealtToChampions,visionScore,...,neutralMinionsKilled.9,magicDamageDealt.9,magicDamageDealtToChampions.9,magicDamageTaken.9,wardsPlaced.9,gameMode.9,gameVersion.9,gameDuration.9,gameId.9,target
187981,33.0,1.0,3.0,0.0,0.985821,1.734062,0.371929,-0.232525,1.446699,0.224572,...,-0.401625,-0.600923,-0.597908,1.053129,0.195191,0.0,31.0,0.751761,0.575520,0.985821
67977,148.0,0.0,2.0,0.0,-1.014383,-0.592623,-0.228340,-0.961786,-1.131530,-0.708928,...,-0.553554,0.052257,0.108406,-1.383624,-0.603164,0.0,10.0,-1.537217,-0.928192,-1.014383
64272,127.0,0.0,2.0,0.0,-1.014383,0.464961,0.672063,0.679051,0.542272,0.224572,...,2.940808,-0.707321,-0.733607,0.306421,-0.503369,0.0,9.0,0.547748,-0.981724,-1.014383
262999,68.0,1.0,2.0,0.0,0.985821,-0.381106,-0.828610,-0.779471,-0.847232,-0.475553,...,-0.553554,-0.324831,-0.100345,-0.440637,-0.303781,0.0,58.0,-1.227835,1.150323,0.985821
120108,72.0,3.0,0.0,0.0,0.985821,0.041927,1.872602,-0.597155,0.279939,-0.522228,...,-0.363643,1.908293,1.143244,1.297120,0.294985,0.0,22.0,1.305510,-0.179026,0.985821
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,126.0,3.0,5.0,0.0,-1.014383,-0.169589,0.371929,0.314421,-0.453524,-0.802277,...,-0.553554,-0.332838,0.401572,-0.448881,-0.203986,0.0,22.0,-0.864648,-0.182143,-1.014383
259178,30.0,2.0,0.0,0.0,0.985821,-0.169589,1.572467,-0.414840,0.283280,-0.382203,...,0.244072,0.891568,1.583788,1.303785,0.095397,0.0,56.0,1.359315,1.123443,0.985821
131932,129.0,0.0,4.0,0.0,-1.014383,0.253444,-1.128744,0.132106,1.264043,-0.708928,...,-0.553554,2.093389,4.007789,1.050147,0.095397,0.0,25.0,0.650875,-0.034144,-1.014383
146867,157.0,2.0,4.0,0.0,-1.014383,-1.015657,0.071794,-0.779471,-0.909203,0.317922,...,2.295111,0.014039,-0.447875,0.385179,0.893751,0.0,28.0,0.059014,0.148665,-1.014383


In [13]:
tabular_model.fit(train=train_df, validation=valid_df)

tabular_model.save_model("model.pth")  # 학습된 모델 저장

Seed set to 42


2024-05-20 11:00:26,539 - {pytorch_tabular.tabular_model:524} - INFO - Preparing the DataLoaders

2024-05-20 11:00:27,963 - {pytorch_tabular.tabular_datamodule:499} - INFO - Setting up the datamodule for          
classification task

c:\Users\a\AppData\Local\pypoetry\Cache\virtualenvs\tabulr_transformers-5PYuSfOC-py3.11\Lib\site-packages\pytorch_tabular\categorical_encoders.py:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_encoded[col].fillna(self._imputed, inplace=True)
c:\Users\a\AppData\Local\pypoetry\Cache\virtualenvs\tabulr_transformers-5PYuSfOC-py3.11\Lib\site-packages\sklearn\utils\extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
c:\Users\a\AppData\Local\pypoetry\C

2024-05-20 11:00:33,973 - {pytorch_tabular.tabular_model:574} - INFO - Preparing the Model: TabTransformerModel

...

c:\Users\a\AppData\Local\pypoetry\Cache\virtualenvs\tabulr_transformers-5PYuSfOC-py3.11\Lib\site-packages\pytorch_l
ightning\trainer\call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...

2024-05-20 11:01:52,281 - {pytorch_tabular.tabular_model:663} - INFO - Training the model completed

2024-05-20 11:01:52,283 - {pytorch_tabular.tabular_model:1489} - INFO - Loading the best model

2024-05-20 11:01:52,284 - {pytorch_tabular.tabular_model:1497} - WARNING - No best model available to load. Did you
run it more than 1 epoch?...

2024-05-20 11:01:52,287 - {pytorch_tabular.tabular_model:1533} - WARNING - Directory is not empty. Overwriting the 
contents.